# Install Required Libraries

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

# Log in to WandB to track finetuning metrics
# Log in to Hugging Face

In [ ]:
!pip install wandb
!wandb login

In [3]:
from huggingface_hub import notebook_login

notebook_login()

# Load & Quantize the model

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Preprocessing the model

In [5]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Import LoRa Config

In [6]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

# Load and preprocessing the dataset

In [7]:
from datasets import load_dataset

# Split the data into 10% test, 10% validation, 80% training
train_data = load_dataset("Hieu-Pham/cooking_squad_splitted", split='train')
val_data = load_dataset("Hieu-Pham/cooking_squad_splitted", split='validation')

def preprocess_dataset(dataset):

    grouped = []

    for row in dataset:
      answers = row['answers']
      input = '[INST] Question: ' + row['question'] + ' Context: ' + row['context'] + ' [/INST]\nAnswer: ' + answers['text']
      grouped.append(input)

    return grouped

input_column = preprocess_dataset(train_data)
train_data = train_data.add_column('Inputs', input_column)


train_data = train_data.map(
    lambda row: tokenizer(row["Inputs"], truncation=True),
    batched=True,
    remove_columns=train_data.column_names,
)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1793 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
# Preprocessing validation dataset
# Validation dataset
output_column = preprocess_dataset(val_data)
val_data = val_data.add_column('Outputs', output_column)

val_data = val_data.map(
    lambda row: tokenizer(row["Outputs"], truncation=True),
    batched=True,
    remove_columns=val_data.column_names,
)

Map:   0%|          | 0/224 [00:00<?, ? examples/s]

# Training Configurations and Start Training

In [9]:
import transformers

# tokenizer.pad_token = tokenizer.eos_token

training_args = transformers.TrainingArguments(
    output_dir="Llama2-7B-QLoRA-cooking-text-gen-prompting",
    evaluation_strategy="steps",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_steps=10,
    logging_steps=20,
    max_grad_norm=0.3,
    max_steps=300,
    warmup_ratio=0.03,
    fp16=True,
    optim="paged_adamw_8bit",
    push_to_hub=True,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings
trainer.train()

wandb: Currently logged in as: hieupham. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
20,1.410400,1.046584
40,0.956900,0.936805
60,0.928600,0.916666
80,0.902800,0.905166
100,0.877200,0.897664
120,0.863900,0.893782
140,0.847800,0.891183
160,0.861100,0.888349
180,0.854700,0.884610
200,0.842300,0.881430


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=300, training_loss=0.9007607587178548, metrics={'train_runtime': 1057.2106, 'train_samples_per_second': 4.54, 'train_steps_per_second': 0.284, 'total_flos': 5.546633446703923e+16, 'train_loss': 0.9007607587178548, 'epoch': 2.67})